In [16]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

# api_key = os.getenv("OPENAI_API_KEY")

if not os.getenv("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")



from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [17]:
from uuid import uuid4

from langchain_core.documents import Document

infinite_max_document_1 = Document(
    page_content="""不只大，要就最大，無限MAX 超跑人體工學椅，我們累計了銷售10,000+的消費者使用體驗，並針對大家最重視的腰部支撐再進化！寬度、高度與厚度同步加大，完全滿足 '' 支撐加倍 X 舒適加倍 ''""",
    metadata={"source": "https://www.marsrhino.com/tw/product_detail?id=54&class_id=13",
              "chair": "infinite_max",
              "max_weight": 150,
              "has_waist_support": True,
              "image_path": "/home/kh/AutoChair/chair_image/infinite_max.jpg"
              }
)

infinite_max_document_2 = Document(
    page_content="""3D ''大'' 體感腰靠，藉由腰靠的高低、前後手動調整，及對應腰部曲線角度自動旋轉，完美支撐腰部，達到舒適與放鬆。""",
    metadata={"source": "https://www.marsrhino.com/tw/product_detail?id=54&class_id=13",
              "chair": "infinite_max",
              "max_weight": 150,
              "has_waist_support": True,
              "image_path": "/home/kh/AutoChair/chair_image/infinite_max.jpg"
              }
)

infinite_max_document_3 = Document(
    page_content="""
                建議身形 身高 : 165 - 200cm,
                最大承重 150kg,
                頭枕 / 椅背 / 椅座 ( 獨創透氣紓壓技術 )
                表層 : 防潑水彈性布
                內層 : 高密度泡棉
                底層 : 透氣網布
                """,
    metadata={"source": "https://www.marsrhino.com/tw/product_detail?id=54&class_id=13",
              "chair": "infinite_max",
              "max_weight": 150,
              "has_waist_support": True,
              "image_path": "/home/kh/AutoChair/chair_image/infinite_max.jpg"
              }
)

In [18]:

documents = [
    infinite_max_document_1,
    infinite_max_document_2,
    infinite_max_document_3
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['02a9066a-1f47-42de-ae6a-0ba09ae4bc1e',
 '35827828-273a-4831-862f-60411d36f619',
 '649a513f-50b2-4ac2-844a-c4ae74657291']

In [19]:
results = vector_store.similarity_search(
    "我想要有大腰靠的椅子",
    k=2,
    filter={"has_waist_support": True},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")
    if res.metadata.get("image_path"):
        from PIL import Image
        Image.open(res.metadata["image_path"]).show()
    print("")


* 不只大，要就最大，無限MAX 超跑人體工學椅，我們累計了銷售10,000+的消費者使用體驗，並針對大家最重視的腰部支撐再進化！寬度、高度與厚度同步加大，完全滿足 '' 支撐加倍 X 舒適加倍 '' [{'source': 'https://www.marsrhino.com/tw/product_detail?id=54&class_id=13', 'chair': 'infinite_max', 'max_weight': 150, 'has_waist_support': True, 'image_path': '/home/kh/AutoChair/chair_image/infinite_max.jpg'}]

* 3D ''大'' 體感腰靠，藉由腰靠的高低、前後手動調整，及對應腰部曲線角度自動旋轉，完美支撐腰部，達到舒適與放鬆。 [{'source': 'https://www.marsrhino.com/tw/product_detail?id=54&class_id=13', 'chair': 'infinite_max', 'max_weight': 150, 'has_waist_support': True, 'image_path': '/home/kh/AutoChair/chair_image/infinite_max.jpg'}]



In [21]:
infinite_max_document_1.metadata["image_path"]

'/home/kh/AutoChair/chair_image/infinite_max.jpg'

In [6]:
results = vector_store.similarity_search(
    "LangChain 提供抽象概念，讓 LLM 的使用變得簡單",
    k=2,
    filter={"source": {"$eq": "tweet"}},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [17]:
results = vector_store.similarity_search_with_score(
    "明天會很熱嗎？", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.850298] 明天的天氣預報為陰天，最高氣溫為 62 度。 [{'source': 'news'}]


In [18]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("偷銀行錢是犯罪行為", filter={"source": "news"})

[Document(id='a3f0e53e-bd3a-455d-b808-e39350a7f63f', metadata={'source': 'news'}, page_content='劫匪闖入市銀行偷走了 100 萬美元現金。')]

In [21]:
vector_store.save_local("chairDB")

new_vector_store = FAISS.load_local(
    "chairDB", 
    embeddings, 
    allow_dangerous_deserialization=True
)

docs = new_vector_store.similarity_search("qux")

In [20]:
docs[0]

Document(id='65a6e4fd-49a0-4480-94aa-005f4e73e4b0', metadata={'source': 'tweet'}, page_content='使用 LangChain 建立一個令人興奮的新專案 - 快來看看吧！')